In [ ]:
import ast
import datetime
import pandas as pd

# Sanity Analysis

In [ ]:
# Load per year tables and check if they have the correct number of columns
base_records_path = "../data/records-"
records_extension = ".csv"

for year in range(1946, 2024):
    records_path = base_records_path + str(year) + records_extension
    records_for_year = pd.read_csv(records_path, index_col='ID', keep_default_na=False)
    col_count = len(records_for_year.columns)
    if col_count != 6:
        print("Year " + str(year) + " has wrong number of columns: " + str(col_count))

In [ ]:
# Load the all records table as the primary data frame
records_path = "../data/records.csv"
df = pd.read_csv(records_path, index_col='ID', keep_default_na=False)

In [ ]:
def checkValues(data, col, ignoreNulls=False, ignoreNotUnique=False):
  hasNull = data[col].isnull().values.any()
  isUnique = data[col].is_unique

  if not ignoreNulls and hasNull:
    print(col + " has null values!")

  if not ignoreNotUnique and not isUnique:
    print(col + " is not unique!")

In [ ]:
# Check for null and non-unique values

if not df.index.is_unique:
    print('IDs are not unique')
    
checkValues(df, 'Body', ignoreNotUnique=True)
checkValues(df, 'Title')
checkValues(df, 'Date', ignoreNotUnique=True)
checkValues(df, 'Resolution')
checkValues(df, 'Subjects', ignoreNotUnique=True)
checkValues(df, 'Voting Data', ignoreNotUnique=True)

In [ ]:
# Check which, if any, rows have duplicate resolutions 
res_counts = df.Resolution.value_counts()
unique_breaking_res = res_counts.index[res_counts.gt(1)]
df[df.Resolution.isin(unique_breaking_res)]

In [ ]:
# Check which, if any, rows have null resolutions
df[df.Resolution.isnull()]

In [ ]:
# Check if 'Body' only contains the two valid values
display(df['Body'].unique())

In [ ]:
# Check if all subjects values can be parsed into sets
for index, row in df.iterrows():
    ast.literal_eval(row['Subjects'])

In [ ]:
# Is voting data in the correct format and are there and duplicate countries?

valid_votes = {'A', 'Y', 'N', 'X'}

for index, row in df.iterrows():
    votingData = row['Voting Data']
    if votingData == 'Concensus' or votingData == 'N/A':
        continue
        
    countries = set()
    
    for votingPoint in votingData.split(';'):
        vote = votingPoint[0]
        country = votingPoint[2:]
        
        if not vote in valid_votes:
            print("Invalid vote choice for ID " + str(index) + ": " + votingPoint)
            
        if not votingPoint[1] == ' ':
            print("Invalid voting data format for ID " + str(index) + ": " + votingPoint)
        
        if country in countries:
            print("Duplicate voting country for ID " + str(index) + ": " + country)
            continue

        countries.add(country)

In [ ]:
# Are all dates in the correct format?
for index, row in df.iterrows():
    datetime.datetime.strptime(row['Date'], '%Y-%m-%d')